# Importing Dataset

In [18]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
df_resume=pd.read_csv('raw_data/Data_Set_CV_Cleaned_version test.csv')
df_resume.drop(['Unnamed: 0'],axis=1,inplace=True)
df_resume=df_resume.rename(columns={'New_Category':'Category','CV_cleaned':'Resume'})
df_resume.dropna(inplace=True)
df_resume=df_resume.drop_duplicates()
df_resume['lenght']=df_resume['Resume'].apply(len)

In [20]:
len_max=10000
df_resume_filtred=df_resume[df_resume['lenght']<len_max]

In [21]:
num_categories= len(df_resume_filtred['Category'].value_counts())
num_categories

15

In [22]:
from sklearn.preprocessing import LabelEncoder
# Convert labels to integers
label_encoder = LabelEncoder()
df_resume_filtred['N_Category'] = label_encoder.fit_transform(df_resume_filtred['Category'])

/var/folders/87/81r_6drs6w508tt7lz6gngyh0000gn/T/ipykernel_3443/1570876403.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resume_filtred['N_Category'] = label_encoder.fit_transform(df_resume_filtred['Category'])


# Logistic Regression Model TF-IDF 

In [23]:
#Division des données en ensembles d'entraînement, de validation et de test
# Création d'un échantillon aléatoire de  du dataset complet
frac=0.2
sample_df = df_resume_filtred.sample(frac=frac, random_state=42)
full_df=df_resume_filtred

df=sample_df

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Division en deux set d'entraînement et de test sur tout le dataset
X=df['Resume']
y=df['N_Category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Création d'une représentation TF-IDF des textes
tfidf_vectorizer = TfidfVectorizer(max_features=50000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Entraînement
logistic_regression_model = LogisticRegression(max_iter=40000, solver='liblinear')

# Définir les paramètres à optimiser
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'max_iter': [100, 1000, 10000, 20000]}

# Utiliser GridSearchCV pour optimiser les paramètres
grid_search = GridSearchCV(logistic_regression_model, param_grid, cv=5)
grid_search.fit(X_train_tfidf, y_train)

# Afficher les meilleurs paramètres
print("Meilleurs paramètres:", grid_search.best_params_)

# Utiliser le modèle avec les meilleurs paramètres
logistic_regression_model = grid_search.best_estimator_


# Évaluation avec validation croisée
cv_scores = cross_val_score(logistic_regression_model, X_train_tfidf, y_train, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))

# Afficher les scores de validation croisée
print("Cross-Validation Scores:", cv_scores)
print("Moyenne des Scores de Validation Croisée:", cv_scores.mean())

#fit the model
logistic_regression_model.fit(X_train_tfidf, y_train)

# Predict
predictions = logistic_regression_model.predict(X_test_tfidf)

# evaluation
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)


Meilleurs paramètres: {'C': 100, 'max_iter': 100}
Cross-Validation Scores: [0.55555556 0.67346939 0.59183673 0.59183673 0.64285714]
Moyenne des Scores de Validation Croisée: 0.611111111111111
Accuracy: 0.5609756097560976
Classification Report:
               precision    recall  f1-score   support

           0       0.33      0.56      0.42         9
           1       0.37      0.70      0.48        10
           2       0.80      0.57      0.67         7
           3       0.61      0.73      0.67        15
           4       0.42      0.83      0.56         6
           5       0.75      0.75      0.75         8
           6       1.00      0.80      0.89        10
           7       0.50      0.25      0.33         4
           8       0.91      0.71      0.80        14
           9       0.00      0.00      0.00        10
          10       0.57      0.67      0.62        12
          11       0.00      0.00      0.00         5
          12       0.00      0.00      0.00         

/Users/hamza/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hamza/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hamza/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

In [27]:
# Sauvegarder le modèle
import joblib

joblib.dump(label_encoder, 'models/label_encoder.joblib')
joblib.dump(logistic_regression_model, 'models/tf_idf_model.joblib')
#joblib.dump(tfidf_model, 'models/tf_idf_model.joblib')
joblib.dump(tfidf_vectorizer, 'models/tfidf_vectorizer.joblib')



['models/tfidf_vectorizer.joblib']